# INFO 4271 - Exercise 3 - Indexing

Issued: April 30, 2024

Due: May 6, 2024

Please submit this filled sheet via Ilias by the due date.

---

# 1. Skip Pointers
Skip pointers can be used to accelerate posting list intersection, allowing pointers to be moved either to the next sequential list position or to the position of the skip pointer if one is available.

a) Implement the `intersect\_skip()` function sketched below. Each time you would ordinarily increment a pointer by one, you can alternatively follow a skip pointer, if one is available at the position.

In [14]:
#Intersect two sorted posting lists that contain skip pointers
def intersect_skip(A, B, use_skip_pointers=True):
    matches = []
    i = 0
    j = 0
    while(i < len(A) and j < len(B)):
        if(A[i][0] == B[j][0]):
            print(f"Found match {A[i][0]}: Increment pointer A from {i} to {i + 1} and B from {j} to {j + 1}")
            matches.append(A[i][0])
            i += 1
            j += 1
        elif A[i] <= B[j]:
            if use_skip_pointers and A[i][1] is not None and A[i][2] <= B[j][0]:
                print(f"A: use skip pointer from {i} to {A[i][1]} (value {A[i][2]})")
                i = A[i][1]
            else:
                print(f"A: Increment pointer from {i} to {i + 1}")
                i += 1
        else:
            if use_skip_pointers and B[j][1] is not None and B[j][2] <= A[i][0]:
                print(f"B: use skip pointer from {j} to {B[j][1]} (value {B[j][2]})")
                j = B[j][1]
            else:
                print(f"B: Increment pointer from {j} to {j + 1}")
                j += 1
    return matches

#Posting lists with skip pointers. 
#Entries take the form [docID, index to skip to, docID at that index]
times_skip = [[2, 3, 16], [12, None, None], [15, None, None], [16, 6, 27], [17, None, None], [23, None, None], [27, None, None]]
square_skip = [[3, 2, 12], [8, None, None], [12, 4, 23], [19, None, None], [23, None, None]]

print(intersect_skip(times_skip, square_skip))

A: Increment pointer from 0 to 1
B: use skip pointer from 0 to 2 (value 12)
Found match 12: Increment pointer A from 1 to 2 and B from 2 to 3
A: Increment pointer from 2 to 3
A: Increment pointer from 3 to 4
A: Increment pointer from 4 to 5
B: Increment pointer from 3 to 4
Found match 23: Increment pointer A from 5 to 6 and B from 4 to 5
[12, 23]


b) How many pointer increment operations did you need to intersect the two posting lists with the given skip pointers? How many operations would it have been for the same lists without skip pointers?

As we can see in the outputs below with skip pointers we need 9 pointer incrementations and without skip pointers we need 11 incrementations.
I counted the following operations as incrementations:
- A pointer was incremented because no match was found and no skip pointer was found
- A match was found and both pointers get increased -> 2 increments

If we would also count using the skip pointers as an increment we would get 10 with skip pointers and 11 without skip pointers.

In [15]:
print(f"With skip pointers:")
intersect_skip(times_skip, square_skip)
print(f"\nWithout skip pointers:")
intersect_skip(times_skip, square_skip, False)

With skip pointers:
A: Increment pointer from 0 to 1
B: use skip pointer from 0 to 2 (value 12)
Found match 12: Increment pointer A from 1 to 2 and B from 2 to 3
A: Increment pointer from 2 to 3
A: Increment pointer from 3 to 4
A: Increment pointer from 4 to 5
B: Increment pointer from 3 to 4
Found match 23: Increment pointer A from 5 to 6 and B from 4 to 5

Without skip pointers:
A: Increment pointer from 0 to 1
B: Increment pointer from 0 to 1
B: Increment pointer from 1 to 2
Found match 12: Increment pointer A from 1 to 2 and B from 2 to 3
A: Increment pointer from 2 to 3
A: Increment pointer from 3 to 4
A: Increment pointer from 4 to 5
B: Increment pointer from 3 to 4
Found match 23: Increment pointer A from 5 to 6 and B from 4 to 5


[12, 23]

# 2. Positional Indices
Positional indices include for each posting the exact positions at which the term can be found in the document. This information allows us to satisfy two previously impossible types of queries. 1) Phrase queries require terms to occur adjacently to one another in a specific order. 2) Range queries allow for more leeway between term positions, merely requiring the two terms to appear within a specified number of tokens.

Implement the `intersect\_range()` function sketched in the code base. Each time you would ordinarily have reported a match, you will now need to check whether the range requirement is satisfied. As an optional addition, think about making this range check efficient using some of the techniques discussed for general posting list intersection.

In [22]:
#Intersect two sorted posting lists with document-internal proximity requirements.
def intersect_range(A, B, range):
    matches = []
    i = 0
    j = 0
    while(i < len(A) and j < len(B)):
        if(A[i][0] == B[j][0]):
            # print(f"{A[i][0]} = {B[j][0]}")
            for posA in A[i][1]:
                for posB in B[j][1]:
                    if abs(posA - posB) <= 3:
                        print(f"Found match {A[i][0]} at posA = {posA} and posB = {posB}: Increment pointer A from {i} to {i + 1} and B from {j} to {j + 1}")
                        matches.append(A[i][0])
            i += 1
            j += 1
        elif A[i] <= B[j]:
            print(f"A: Increment pointer from {i} to {i + 1}")
            i += 1
        else:
            print(f"B: Increment pointer from {j} to {j + 1}")
            j += 1
    return matches

#Posting lists with document-internal positional information.
#Entries take the form [docID, [pos1, pos2, ...]]
times_range = [[2, [15, 128]], [12, [6, 45, 89, 942]], [15, [13]], [16, [1276, 1500]], [17, [13, 89, 90]], [23, [17, 64]], [27, [456, 629]]]
square_range = [[3, [65, 90]], [8, [67, 94]], [12, [3]], [19, [18, 81, 1881]], [23, [63]]]

print(intersect_range(times_range, square_range, 3))

A: Increment pointer from 0 to 1
B: Increment pointer from 0 to 1
B: Increment pointer from 1 to 2
Found match 12 at posA = 6 and posB = 3: Increment pointer A from 1 to 2 and B from 2 to 3
A: Increment pointer from 2 to 3
A: Increment pointer from 3 to 4
A: Increment pointer from 4 to 5
B: Increment pointer from 3 to 4
Found match 23 at posA = 64 and posB = 63: Increment pointer A from 5 to 6 and B from 4 to 5
[12, 23]


# 3. Paper Pick
Don't forget to submit your paper pick at https://forms.gle/JNnd1V7k1L3uPVxU6.